In [6]:

from pathlib import Path
from io import StringIO
import pandas as pd
import numpy as np
import re
import sys

In [7]:
# --- Config: change if your files are elsewhere ---
SRC_DIR = Path('Data')
SRC_WIKI = SRC_DIR / 'Average_height_by_country_Wikipedia.csv'
SRC_DISABLED = SRC_DIR / 'Disabled_World.csv'
SRC_NCD = SRC_DIR / 'Ncd_risc.csv'
OUT_DIR = Path('Data'); OUT_DIR.mkdir(exist_ok=True)
OUT_CSV = OUT_DIR / 'average_height_per_country.csv'



In [8]:
# --- Helpers ---
def read_csv_try(path: Path):
    """Try a few encodings, fall back to latin-1 replacement."""
    encs = ['utf-8', 'latin-1', 'cp1252']
    for enc in encs:
        try:
            return pd.read_csv(path, encoding=enc)
        except Exception:
            pass
    raw = path.read_bytes()
    text = raw.decode('latin-1', errors='replace')
    return pd.read_csv(StringIO(text))

def find_col(df: pd.DataFrame, keywords):
    """Return first column name from df whose lowercased header contains any keyword substring."""
    if df is None:
        return None
    cols = list(df.columns)
    for kw in keywords:
        kl = kw.lower()
        for c in cols:
            if kl in str(c).lower():
                return c
    return None

def to_cm(val):
    """Parse a height string/number and return value in centimeters or NaN."""
    if pd.isna(val):
        return np.nan
    s = str(val).strip().replace(',', '')
    # meters like "1.75 m"
    m = re.search(r'([0-9]+(?:\.[0-9]+)?)\s*m\b', s, flags=re.I)
    if m:
        try:
            return float(m.group(1)) * 100
        except:
            return np.nan
    # centimeters like "175 cm"
    cm = re.search(r'([0-9]+(?:\.[0-9]+)?)\s*cm\b', s, flags=re.I)
    if cm:
        try:
            return float(cm.group(1))
        except:
            return np.nan
    # naked number: >3 => cm else meters
    num = re.search(r'([0-9]+(?:\.[0-9]+)?)', s)
    if num:
        v = float(num.group(1))
        return v if v > 3 else v * 100
    return np.nan



In [9]:
# --- Extractors ---
def extract_from_wiki(df):
    """Extract country/year/sex/height_raw rows from wiki-like table."""
    if df is None or df.shape[0] == 0:
        return pd.DataFrame(columns=['country','year','sex','height_raw'])
    country_col = find_col(df, ['country', 'country / region', 'region'])
    year_col = find_col(df, ['year'])
    male_col = find_col(df, ['male', 'average male', 'male height'])
    female_col = find_col(df, ['female', 'average female', 'female height'])
    avg_col = find_col(df, ['average height', 'average', 'mean', 'avg'])
    out_rows = []
    for _, r in df.iterrows():
        ctry = r.get(country_col) if country_col in df.columns else None
        year = r.get(year_col) if (year_col and year_col in df.columns) else np.nan
        if male_col and male_col in df.columns and pd.notna(r.get(male_col)):
            out_rows.append({'country': ctry, 'year': year, 'sex': 'male', 'height_raw': r.get(male_col)})
        if female_col and female_col in df.columns and pd.notna(r.get(female_col)):
            out_rows.append({'country': ctry, 'year': year, 'sex': 'female', 'height_raw': r.get(female_col)})
        if (not (male_col or female_col)) and avg_col and avg_col in df.columns and pd.notna(r.get(avg_col)):
            out_rows.append({'country': ctry, 'year': year, 'sex': 'both', 'height_raw': r.get(avg_col)})
    return pd.DataFrame(out_rows)

def extract_from_disabled(df):
    """Extract from Disabled World style table (country, height, maybe sex)."""
    if df is None or df.shape[0] == 0:
        return pd.DataFrame(columns=['country','year','sex','height_raw'])
    country_col = find_col(df, ['country'])
    height_col = find_col(df, ['average height', 'height', 'avg height'])
    sex_col = find_col(df, ['sex', 'gender'])
    year_col = find_col(df, ['year'])
    rows = []
    if height_col and height_col in df.columns:
        for _, r in df.iterrows():
            ctry = r.get(country_col) if (country_col and country_col in df.columns) else None
            year = r.get(year_col) if (year_col and year_col in df.columns) else np.nan
            sex = r.get(sex_col) if (sex_col and sex_col in df.columns) else 'both'
            rows.append({'country': ctry, 'year': year, 'sex': str(sex).strip().lower(), 'height_raw': r.get(height_col)})
    else:
        # fallback: look for male/female numeric columns
        male_col = find_col(df, ['male'])
        female_col = find_col(df, ['female'])
        for _, r in df.iterrows():
            ctry = r.get(country_col) if (country_col and country_col in df.columns) else None
            if male_col and male_col in df.columns and pd.notna(r.get(male_col)):
                rows.append({'country': ctry, 'year': np.nan, 'sex': 'male', 'height_raw': r.get(male_col)})
            if female_col and female_col in df.columns and pd.notna(r.get(female_col)):
                rows.append({'country': ctry, 'year': np.nan, 'sex': 'female', 'height_raw': r.get(female_col)})
    return pd.DataFrame(rows)

def extract_from_ncd(df):
    """Extract from NCD-RISC style table (Sex, Year, Mean height, maybe Age group)."""
    if df is None or df.shape[0] == 0:
        return pd.DataFrame(columns=['country','year','sex','height_raw'])
    sex_col = find_col(df, ['sex'])
    year_col = find_col(df, ['year'])
    mean_col = find_col(df, ['mean height', 'mean', 'mean_height', 'mean height'])
    country_col = find_col(df, ['country', 'location', 'region'])
    rows = []
    if mean_col and mean_col in df.columns:
        for _, r in df.iterrows():
            ctry = r.get(country_col) if (country_col and country_col in df.columns) else None
            year = r.get(year_col) if (year_col and year_col in df.columns) else np.nan
            sex = r.get(sex_col) if (sex_col and sex_col in df.columns) else 'both'
            rows.append({'country': ctry, 'year': year, 'sex': str(sex).strip().lower(), 'height_raw': r.get(mean_col)})
    else:
        # fallback: numeric columns
        numeric_cols = [c for c in df.columns if pd.api.types.is_numeric_dtype(df[c])]
        if numeric_cols:
            col0 = numeric_cols[0]
            for _, r in df.iterrows():
                ctry = r.get(country_col) if (country_col and country_col in df.columns) else None
                rows.append({'country': ctry, 'year': np.nan, 'sex': 'both', 'height_raw': r.get(col0)})
    return pd.DataFrame(rows)



In [11]:
# --- Main flow ---
def main():
    # Verify files exist
    for p in (SRC_WIKI, SRC_DISABLED, SRC_NCD):
        if not p.exists():
            print(f"ERROR: source file not found: {p}", file=sys.stderr)
            # continue to attempt others, but user should check paths
    wiki = read_csv_try(SRC_WIKI) if SRC_WIKI.exists() else pd.DataFrame()
    disabled = read_csv_try(SRC_DISABLED) if SRC_DISABLED.exists() else pd.DataFrame()
    ncd = read_csv_try(SRC_NCD) if SRC_NCD.exists() else pd.DataFrame()
    print("Loaded rows (wiki, disabled, ncd):", len(wiki), len(disabled), len(ncd))

    # Extract
    w = extract_from_wiki(wiki)
    d = extract_from_disabled(disabled)
    n = extract_from_ncd(ncd)
    print("Extracted rows (wiki, disabled, ncd):", len(w), len(d), len(n))

    combined = pd.concat([w, d, n], ignore_index=True)
    # Ensure country column exists and trim
    if 'country' not in combined.columns:
        combined['country'] = None
    combined['country'] = combined['country'].astype(str).str.strip()
    # Drop entries without height_raw
    combined = combined.dropna(subset=['height_raw'])
    # Parse heights
    combined['height_cm'] = combined['height_raw'].apply(to_cm)
    combined = combined.dropna(subset=['height_cm'])
    combined['sex'] = combined['sex'].fillna('both').astype(str).str.lower()
    combined['year'] = pd.to_numeric(combined['year'], errors='coerce').astype('Int64')
    print("After parsing & dropping missing heights:", len(combined))

    # Aggregate
    final = (combined
             .groupby(['country', 'year', 'sex'], dropna=False)
             .agg(average_height_cm=('height_cm', 'mean'), sources_count=('height_cm', 'count'))
             .reset_index()
             .sort_values(['country', 'year', 'sex'])
             .reset_index(drop=True))

    print("Final rows before save:", len(final))
    final.to_csv(OUT_CSV, index=False)
    print("Saved cleaned CSV to:", OUT_CSV)

if __name__ == '__main__':
    main()

Loaded rows (wiki, disabled, ncd): 240 232 1050
Extracted rows (wiki, disabled, ncd): 478 232 1050
After parsing & dropping missing heights: 1667
Final rows before save: 581
Saved cleaned CSV to: Data\average_height_per_country.csv
